Some notes: 
    
    three input vectors (or tensors if one-hot encoded) for the candidate names, the station callsigns and the committee names

In [1]:
#!pip install tensorflow
#!pip install keras


In [2]:
from __future__ import print_function
import pandas as pd


from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
train = "source/training.csv"
filings = "source/ftf-all-filings.tsv"
docs = 100000

In [4]:
dft = pd.read_csv(train, nrows = docs)
dft.head()

,slug,page,x0,y0,x1,y1,token,gross_amount
0,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,423.000,25.145,439.448,25.189,Print,0.00
1,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,441.672,25.145,458.568,25.189,Date,0.00
2,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,474.001,25.145,505.137,25.189,10/09/12,0.40
3,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,533.251,25.145,551.931,25.189,Page,0.00
4,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,560.251,25.145,564.699,25.189,1,0.25


In [5]:
dff = pd.read_csv(filings,nrows = docs, sep='\t')
dff.head()


,id,filing_type,contract_number,url,committee,agency,callsign,dc_slug,thumbnail_url,gross_amount,market_id,upload_date
0,23969,Federal,314271,/collect/files/59438/Political File/2012/Feder...,NRSC-MANDEL/REPUBLICAN,STRATEGIC MEDIA SERVICES,WCPO-TV,459790-nrsc_mandel_rep_senate_1010_101412_314271,https://s3.amazonaws.com/s3.documentcloud.org/...,21100.0,35.0,2012-10-10 00:00:00 UTC
1,2333,Non-Candidate Issue Ads,9792042,/collect/files/11289/Political File/2012/Non-C...,60+ ASSOCIATION,MENTZER MEDIA SERVICES,WKRC-TV,416491-collect-files-11289-political-file-2012...,https://s3.amazonaws.com/s3.documentcloud.org/...,6250.0,35.0,2012-08-14 00:00:00 UTC
2,31990,Federal,336298,/collect/files/73910/Political File/2012/Feder...,POL/R TERRY/D/PRES/US,KATHLEEN OFFERMAN,WPXI,470297-101612-terry-contract-2-13504017232906-...,https://s3.amazonaws.com/s3.documentcloud.org/...,30.0,23.0,2012-10-16 00:00:00 UTC
3,33211,Federal,116252,/collect/files/73123/Political File/2012/Feder...,COMMITTEE TO ELECT SHANELLE JACKSON,SHANELLE JACKSON FOR CONGRESS,WJBK,473630-116252-0-13442821773323-_-pdf,https://s3.amazonaws.com/s3.documentcloud.org/...,1170.0,11.0,2012-08-06 00:00:00 UTC
4,6213,Non-Candidate Issue Ads,48982,/collect/files/72313/Political File/2012/Non-C...,PRIORITIES USA ACTION,MUNDY KATOWITZ MEDIA,WHP-TV,420426-collect-files-72313-political-file-2012...,https://s3.amazonaws.com/s3.documentcloud.org/...,9335.0,41.0,2012-08-13 00:00:00 UTC


Useful Bash Code: 
    
    
    head -n 50 training.csv
    head -n 30 ftf-all-filings.tsv

In [6]:
df_all = pd.merge(left=dft, right=dff, how='left', left_on='slug', right_on='dc_slug')
#df_all = df_all[['slug', 'page', 'x0', 'y0', 'x1', 'y1', 'token', 'gross_amount_x', 'committee']]
df_all = df_all[['slug', 'token', 'committee']]

df_all.head(10)

,slug,token,committee
0,459790-nrsc_mandel_rep_senate_1010_101412_314271,Print,NRSC-MANDEL/REPUBLICAN
1,459790-nrsc_mandel_rep_senate_1010_101412_314271,Date,NRSC-MANDEL/REPUBLICAN
2,459790-nrsc_mandel_rep_senate_1010_101412_314271,10/09/12,NRSC-MANDEL/REPUBLICAN
3,459790-nrsc_mandel_rep_senate_1010_101412_314271,Page,NRSC-MANDEL/REPUBLICAN
4,459790-nrsc_mandel_rep_senate_1010_101412_314271,1,NRSC-MANDEL/REPUBLICAN
5,459790-nrsc_mandel_rep_senate_1010_101412_314271,of,NRSC-MANDEL/REPUBLICAN
6,459790-nrsc_mandel_rep_senate_1010_101412_314271,3,NRSC-MANDEL/REPUBLICAN
7,459790-nrsc_mandel_rep_senate_1010_101412_314271,WCPO,NRSC-MANDEL/REPUBLICAN
8,459790-nrsc_mandel_rep_senate_1010_101412_314271,ORDER,NRSC-MANDEL/REPUBLICAN
9,459790-nrsc_mandel_rep_senate_1010_101412_314271,Flight,NRSC-MANDEL/REPUBLICAN


In [7]:
df_all['slug'][0]

'459790-nrsc_mandel_rep_senate_1010_101412_314271'

In [8]:
df_group = df_all.groupby(['slug','committee'])['token'].apply(lambda a: ' '.join([str(x) for x in a])).reset_index()
df_group.head(50)

,slug,committee,token
0,416903-collect-files-39738-political-file-2012...,60+ ASSOCIATION,Print Date 08/14/12 Page 1 of 1 WXIX ORDER Fli...
1,416904-collect-files-39738-political-file-2012...,60+ ASSOCIATION,Print Date 08/14/12 Page 1 of 1 WXIX ORDER Fli...
2,416945-collect-files-59441-political-file-2012...,60+ ASSOCIATION,Print Date 08/17/12 Page 1 of 2 WEWS ORDER Fli...
3,416946-collect-files-59441-political-file-2012...,60+ ASSOCIATION,Print Date 08/17/12 Page 1 of 2 WEWS ORDER Fli...
4,417116-collect-files-65593-political-file-2012...,RNC,Print Date 08/10/12 18:15:03 Page 1 of 6 ORDER...
5,419754-collect-files-65684-political-file-2012...,WARREN/D/SENATE,Contract Agreement Between: Print Date 08/23/1...
6,419756-collect-files-65684-political-file-2012...,WARREN/D/SENATE,Contract Agreement Between: Print Date 08/23/1...
7,419761-collect-files-65684-political-file-2012...,WARREN/D/SENATE,Contract Agreement Between: Print Date 08/29/1...
8,419765-collect-files-65684-political-file-2012...,WARREN/D/SENATE,Contract Agreement Between: Print Date 08/09/1...
9,420232-collect-files-39746-political-file-2012...,60+ ASSOCIATION,Contract Agreement Between: Print Date 08/14/1...


In [9]:
print(df_group['slug'][0])

' '.join(df_all[df_all['slug'] == '416903-collect-files-39738-political-file-2012-non']['token'])

416903-collect-files-39738-political-file-2012-non


'Print Date 08/14/12 Page 1 of 1 WXIX ORDER Flight Dates08/15/12-08/21/12 Order Sep 00:00:00 Product Estimate # 1224 Contract / Revision 808557 / 60 PLUS ASSOC SENATE Alt Order # 06276486 Original Date / Revision Agency Com 15% Billing Type Cash 08/14/12 08/14/12 Order Type Political Billing Contact Billing Cycle EOM/EOC Advertiser60 Plus Assoc 600 Farimont Avenue Billing CalendarBROADCAST Suite 306 Demographic A35+ Towson, MD 21286 Agency Mentzer Media Services Rev Codes Agency Political Pol-Issue Buying Contact Sales Office T-DC Product Codes PL20 Sales Region National Priority P 1 600 Farimont Avenue Suite 306 Agency Ref Advertiser Ref Towson, MD 21286 Primary Account Executive Aaron Ashe Account Executive Order% Start Date End Date Aaron Ashe 100% Order Share % Market Value Competing Station % of Order Amount EXIX % WBQC % WCET % WCPO % WCVN % WKON % WKRC % WLWT % WPTO % WSTR % Order Totals Billing Plan Month # of Spots Net Amount Gross Amount Rating Start Date End Date # Spots Net

https://www.tensorflow.org/guide/migrate to convert TF1 code to TF2

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


# Applying this method to our data: 

In [10]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

In [11]:
input_texts = []
input_texts = df_group['token'][:num_samples]

input_characters = set()
for token in input_texts:
        for char in token:
            if char not in input_characters:
                input_characters.add(char)
            if ' ' not in input_characters: 
                input_characters.add(' ')
        if '\t' not in input_characters: 
            input_characters.add('\t')   

In [12]:
target_texts = []
target_texts = df_group['committee'][:num_samples]

target_characters = set()
for token in target_texts:
    if token != 'nan':
        for char in token:
            if char not in target_characters:
                target_characters.add(char)
        if ' ' not in target_characters: 
            target_characters.add(' ')
        if '\t' not in target_characters: 
            target_characters.add('\t')        
        


In [13]:
input_characters = list(input_characters)
target_characters = list(target_characters)
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

#print(input_texts[:10])
#print(input_characters)

Number of unique input tokens: 87
Number of unique output tokens: 35
Max sequence length for inputs: 78045
Max sequence length for outputs: 86


In [14]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
print (input_token_index)
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

{'4': 0, 'F': 1, 'c': 2, 'd': 3, 'A': 4, ')': 5, '9': 6, 'g': 7, 'M': 8, ']': 9, 'U': 10, '?': 11, 'B': 12, 'O': 13, 'e': 14, 'r': 15, 'i': 16, 'X': 17, '.': 18, '=': 19, 'R': 20, '3': 21, 'H': 22, 'u': 23, 'Q': 24, '<': 25, '&': 26, 'V': 27, 'W': 28, '%': 29, '"': 30, 'h': 31, 'N': 32, '2': 33, 'v': 34, ',': 35, "'": 36, 'L': 37, 'y': 38, '/': 39, '[': 40, '’': 41, '8': 42, '0': 43, 's': 44, 't': 45, 'f': 46, '6': 47, 'p': 48, 'm': 49, 'P': 50, ':': 51, '+': 52, 'Z': 53, '\t': 54, 'J': 55, '5': 56, '(': 57, '#': 58, 'G': 59, 'x': 60, ';': 61, 'k': 62, 'b': 63, 'T': 64, '-': 65, 'I': 66, 'o': 67, '@': 68, 'a': 69, 'C': 70, 'K': 71, 'z': 72, 'j': 73, 'q': 74, 'n': 75, '*': 76, 'w': 77, 'l': 78, 'E': 79, '1': 80, '$': 81, ' ': 82, 'S': 83, 'Y': 84, 'D': 85, '7': 86}


In [15]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        #decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [16]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [17]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)



In [19]:
print(decoder_outputs)

Tensor("dense/Identity:0", shape=(None, None, 35), dtype=float32)


In [20]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 44 samples, validate on 11 samples
Epoch 1/10


In [ ]:
# Save model
model.save('s2s_politics1.h5')

In [ ]:

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

In [ ]:
# '''
# #Sequence to sequence example in Keras (character-level).
# This script demonstrates how to implement a basic character-level
# sequence-to-sequence model. We apply it to translating
# short English sentences into short French sentences,
# character-by-character. Note that it is fairly unusual to
# do character-level machine translation, as word-level
# models are more common in this domain.
# **Summary of the algorithm**
# - We start with input sequences from a domain (e.g. English sentences)
#     and corresponding target sequences from another domain
#     (e.g. French sentences).
# - An encoder LSTM turns input sequences to 2 state vectors
#     (we keep the last LSTM state and discard the outputs).
# - A decoder LSTM is trained to turn the target sequences into
#     the same sequence but offset by one timestep in the future,
#     a training process called "teacher forcing" in this context.
#     It uses as initial state the state vectors from the encoder.
#     Effectively, the decoder learns to generate `targets[t+1...]`
#     given `targets[...t]`, conditioned on the input sequence.
# - In inference mode, when we want to decode unknown input sequences, we:
#     - Encode the input sequence into state vectors
#     - Start with a target sequence of size 1
#         (just the start-of-sequence character)
#     - Feed the state vectors and 1-char target sequence
#         to the decoder to produce predictions for the next character
#     - Sample the next character using these predictions
#         (we simply use argmax). 
#     - Append the sampled character to the target sequence
#     - Repeat until we generate the end-of-sequence character or we
#         hit the character limit.
# **Data download**
# [English to French sentence pairs.
# ](http://www.manythings.org/anki/fra-eng.zip)
# [Lots of neat sentence pairs datasets.
# ](http://www.manythings.org/anki/)
# **References**
# - [Sequence to Sequence Learning with Neural Networks
#    ](https://arxiv.org/abs/1409.3215)
# - [Learning Phrase Representations using
#     RNN Encoder-Decoder for Statistical Machine Translation
#     ](https://arxiv.org/abs/1406.1078)
# '''

# batch_size = 64  # Batch size for training.
# epochs = 100  # Number of epochs to train for.
# latent_dim = 256  # Latent dimensionality of the encoding space.
# num_samples = 10000  # Number of samples to train on.
# # Path to the data txt file on disk.
# data_path = 'fra-eng/fra.txt'

# # Vectorize the data.
# input_texts = []
# target_texts = []
# input_characters = set()
# target_characters = set()
# with open(data_path, 'r', encoding='utf-8') as f:
#     lines = f.read().split('\n')
# for line in lines[: min(num_samples, len(lines) - 1)]:
#     input_text, target_text, _ = line.split('\t')
#     # We use "tab" as the "start sequence" character
#     # for the targets, and "\n" as "end sequence" character.
#     target_text = '\t' + target_text + '\n'
#     input_texts.append(input_text)
#     target_texts.append(target_text)
#     for char in input_text:
#         if char not in input_characters:
#             input_characters.add(char)
#     for char in target_text:
#         if char not in target_characters:
#             target_characters.add(char)

# input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
# num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])

# print('Number of samples:', len(input_texts))
# print('Number of unique input tokens:', num_encoder_tokens)
# print('Number of unique output tokens:', num_decoder_tokens)
# print('Max sequence length for inputs:', max_encoder_seq_length)
# print('Max sequence length for outputs:', max_decoder_seq_length)

# input_token_index = dict(
#     [(char, i) for i, char in enumerate(input_characters)])
# print(input_token_index)

In [ ]:

# target_token_index = dict(
#     [(char, i) for i, char in enumerate(target_characters)])

# encoder_input_data = np.zeros(
#     (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
#     dtype='float32')
# decoder_input_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
#     dtype='float32')
# decoder_target_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
#     dtype='float32')

# for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
#     for t, char in enumerate(input_text):
#         encoder_input_data[i, t, input_token_index[char]] = 1.
#     encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
#     for t, char in enumerate(target_text):
#         # decoder_target_data is ahead of decoder_input_data by one timestep
#         decoder_input_data[i, t, target_token_index[char]] = 1.
#         if t > 0:
#             # decoder_target_data will be ahead by one timestep
#             # and will not include the start character.
#             decoder_target_data[i, t - 1, target_token_index[char]] = 1.
#     decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
#     decoder_target_data[i, t:, target_token_index[' ']] = 1.
# # Define an input sequence and process it.
# encoder_inputs = Input(shape=(None, num_encoder_tokens))
# encoder = LSTM(latent_dim, return_state=True)
# encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# # We discard `encoder_outputs` and only keep the states.
# encoder_states = [state_h, state_c]

# # Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(None, num_decoder_tokens))
# # We set up our decoder to return full output sequences,
# # and to return internal states as well. We don't use the
# # return states in the training model, but we will use them in inference.
# decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
#                                      initial_state=encoder_states)
# decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

# # Define the model that will turn
# # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# # Run training
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
#               metrics=['accuracy'])
# model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)
# # Save model
# model.save('s2s.h5')

# # Next: inference mode (sampling).
# # Here's the drill:
# # 1) encode input and retrieve initial decoder state
# # 2) run one step of decoder with this initial state
# # and a "start of sequence" token as target.
# # Output will be the next target token
# # 3) Repeat with the current target token and current states

# # Define sampling models
# encoder_model = Model(encoder_inputs, encoder_states)

# decoder_state_input_h = Input(shape=(latent_dim,))
# decoder_state_input_c = Input(shape=(latent_dim,))
# decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# decoder_outputs, state_h, state_c = decoder_lstm(
#     decoder_inputs, initial_state=decoder_states_inputs)
# decoder_states = [state_h, state_c]
# decoder_outputs = decoder_dense(decoder_outputs)
# decoder_model = Model(
#     [decoder_inputs] + decoder_states_inputs,
#     [decoder_outputs] + decoder_states)

# # Reverse-lookup token index to decode sequences back to
# # something readable.
# reverse_input_char_index = dict(
#     (i, char) for char, i in input_token_index.items())
# reverse_target_char_index = dict(
#     (i, char) for char, i in target_token_index.items())


# def decode_sequence(input_seq):
#     # Encode the input as state vectors.
#     states_value = encoder_model.predict(input_seq)

#     # Generate empty target sequence of length 1.
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     # Populate the first character of target sequence with the start character.
#     target_seq[0, 0, target_token_index['\t']] = 1.

#     # Sampling loop for a batch of sequences
#     # (to simplify, here we assume a batch of size 1).
#     stop_condition = False
#     decoded_sentence = ''
#     while not stop_condition:
#         output_tokens, h, c = decoder_model.predict(
#             [target_seq] + states_value)

#         # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_char = reverse_target_char_index[sampled_token_index]
#         decoded_sentence += sampled_char

#         # Exit condition: either hit max length
#         # or find stop character.
#         if (sampled_char == '\n' or
#            len(decoded_sentence) > max_decoder_seq_length):
#             stop_condition = True

#         # Update the target sequence (of length 1).
#         target_seq = np.zeros((1, 1, num_decoder_tokens))
#         target_seq[0, 0, sampled_token_index] = 1.

#         # Update states
#         states_value = [h, c]

#     return decoded_sentence


# for seq_index in range(100):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index: seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     print('-')
#     print('Input sentence:', input_texts[seq_index])
#     print('Decoded sentence:', decoded_sentence)

In [ ]:
# batch_size = 64  # Batch size for training.
# epochs = 100  # Number of epochs to train for.
# latent_dim = 256  # Latent dimensionality of the encoding space.
# num_samples = 10000  # Number of samples to train on.
# # Path to the data txt file on disk.
# data_path = 'fra-eng/fra.txt'

# # Vectorize the data.
# input_texts = []
# target_texts = []
# input_characters = set()
# target_characters = set()
# with open(data_path, 'r', encoding='utf-8') as f:
#     lines = f.read().split('\n')
# for line in lines[: min(num_samples, len(lines) - 1)]:
#     input_text, target_text, _ = line.split('\t')
#     # We use "tab" as the "start sequence" character
#     # for the targets, and "\n" as "end sequence" character.
#     target_text = '\t' + target_text + '\n'
#     input_texts.append(input_text)
#     target_texts.append(target_text)
#     for char in input_text:
#         if char not in input_characters:
#             input_characters.add(char)
#     for char in target_text:
#         if char not in target_characters:
#             target_characters.add(char)

# input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
# num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])

# print('Number of samples:', len(input_texts))
# print('Number of unique input tokens:', num_encoder_tokens)
# print('Number of unique output tokens:', num_decoder_tokens)
# print('Max sequence length for inputs:', max_encoder_seq_length)
# print('Max sequence length for outputs:', max_decoder_seq_length)

# input_token_index = dict(
#     [(char, i) for i, char in enumerate(input_characters)])
# target_token_index = dict(
#     [(char, i) for i, char in enumerate(target_characters)])

In [ ]:
# print(input_characters[:10])

In [ ]:
# '''
# #Sequence to sequence example in Keras (character-level).
# This script demonstrates how to implement a basic character-level
# sequence-to-sequence model. We apply it to translating
# short English sentences into short French sentences,
# character-by-character. Note that it is fairly unusual to
# do character-level machine translation, as word-level
# models are more common in this domain.
# **Summary of the algorithm**
# - We start with input sequences from a domain (e.g. English sentences)
#     and corresponding target sequences from another domain
#     (e.g. French sentences).
# - An encoder LSTM turns input sequences to 2 state vectors
#     (we keep the last LSTM state and discard the outputs).
# - A decoder LSTM is trained to turn the target sequences into
#     the same sequence but offset by one timestep in the future,
#     a training process called "teacher forcing" in this context.
#     It uses as initial state the state vectors from the encoder.
#     Effectively, the decoder learns to generate `targets[t+1...]`
#     given `targets[...t]`, conditioned on the input sequence.
# - In inference mode, when we want to decode unknown input sequences, we:
#     - Encode the input sequence into state vectors
#     - Start with a target sequence of size 1
#         (just the start-of-sequence character)
#     - Feed the state vectors and 1-char target sequence
#         to the decoder to produce predictions for the next character
#     - Sample the next character using these predictions
#         (we simply use argmax).
#     - Append the sampled character to the target sequence
#     - Repeat until we generate the end-of-sequence character or we
#         hit the character limit.
# **Data download**
# [English to French sentence pairs.
# ](http://www.manythings.org/anki/fra-eng.zip)
# [Lots of neat sentence pairs datasets.
# ](http://www.manythings.org/anki/)
# **References**
# - [Sequence to Sequence Learning with Neural Networks
#    ](https://arxiv.org/abs/1409.3215)
# - [Learning Phrase Representations using
#     RNN Encoder-Decoder for Statistical Machine Translation
#     ](https://arxiv.org/abs/1406.1078)
# '''

# batch_size = 64  # Batch size for training.
# epochs = 100  # Number of epochs to train for.
# latent_dim = 256  # Latent dimensionality of the encoding space.
# num_samples = 10000  # Number of samples to train on.
# # Path to the data txt file on disk.
# data_path = 'fra-eng/fra.txt'

# # Vectorize the data.
# input_texts = []
# target_texts = []
# input_characters = set()
# target_characters = set()
# with open(data_path, 'r', encoding='utf-8') as f:
#     lines = f.read().split('\n')
# for line in lines[: min(num_samples, len(lines) - 1)]:
#     input_text, target_text, _ = line.split('\t')
#     # We use "tab" as the "start sequence" character
#     # for the targets, and "\n" as "end sequence" character.
#     target_text = '\t' + target_text + '\n'
#     input_texts.append(input_text)
#     target_texts.append(target_text)
#     for char in input_text:
#         if char not in input_characters:
#             input_characters.add(char)
#     for char in target_text:
#         if char not in target_characters:
#             target_characters.add(char)

# input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
# num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])

# print('Number of samples:', len(input_texts))
# print('Number of unique input tokens:', num_encoder_tokens)
# print('Number of unique output tokens:', num_decoder_tokens)
# print('Max sequence length for inputs:', max_encoder_seq_length)
# print('Max sequence length for outputs:', max_decoder_seq_length)

# input_token_index = dict(
#     [(char, i) for i, char in enumerate(input_characters)])
# target_token_index = dict(
#     [(char, i) for i, char in enumerate(target_characters)])

# encoder_input_data = np.zeros(
#     (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
#     dtype='float32')
# decoder_input_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
#     dtype='float32')
# decoder_target_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
#     dtype='float32')

# for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
#     for t, char in enumerate(input_text):
#         encoder_input_data[i, t, input_token_index[char]] = 1.
#     encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
#     for t, char in enumerate(target_text):
#         # decoder_target_data is ahead of decoder_input_data by one timestep
#         decoder_input_data[i, t, target_token_index[char]] = 1.
#         if t > 0:
#             # decoder_target_data will be ahead by one timestep
#             # and will not include the start character.
#             decoder_target_data[i, t - 1, target_token_index[char]] = 1.
#     decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
#     decoder_target_data[i, t:, target_token_index[' ']] = 1.
# # Define an input sequence and process it.
# encoder_inputs = Input(shape=(None, num_encoder_tokens))
# encoder = LSTM(latent_dim, return_state=True)
# encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# # We discard `encoder_outputs` and only keep the states.
# encoder_states = [state_h, state_c]

# # Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(None, num_decoder_tokens))
# # We set up our decoder to return full output sequences,
# # and to return internal states as well. We don't use the
# # return states in the training model, but we will use them in inference.
# decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
#                                      initial_state=encoder_states)
# decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

# # Define the model that will turn
# # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# # Run training
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
#               metrics=['accuracy'])
# model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)
# # Save model
# model.save('s2s.h5')

# # Next: inference mode (sampling).
# # Here's the drill:
# # 1) encode input and retrieve initial decoder state
# # 2) run one step of decoder with this initial state
# # and a "start of sequence" token as target.
# # Output will be the next target token
# # 3) Repeat with the current target token and current states

# # Define sampling models
# encoder_model = Model(encoder_inputs, encoder_states)

# decoder_state_input_h = Input(shape=(latent_dim,))
# decoder_state_input_c = Input(shape=(latent_dim,))
# decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# decoder_outputs, state_h, state_c = decoder_lstm(
#     decoder_inputs, initial_state=decoder_states_inputs)
# decoder_states = [state_h, state_c]
# decoder_outputs = decoder_dense(decoder_outputs)
# decoder_model = Model(
#     [decoder_inputs] + decoder_states_inputs,
#     [decoder_outputs] + decoder_states)

# # Reverse-lookup token index to decode sequences back to
# # something readable.
# reverse_input_char_index = dict(
#     (i, char) for char, i in input_token_index.items())
# reverse_target_char_index = dict(
#     (i, char) for char, i in target_token_index.items())


# def decode_sequence(input_seq):
#     # Encode the input as state vectors.
#     states_value = encoder_model.predict(input_seq)

#     # Generate empty target sequence of length 1.
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     # Populate the first character of target sequence with the start character.
#     target_seq[0, 0, target_token_index['\t']] = 1.

#     # Sampling loop for a batch of sequences
#     # (to simplify, here we assume a batch of size 1).
#     stop_condition = False
#     decoded_sentence = ''
#     while not stop_condition:
#         output_tokens, h, c = decoder_model.predict(
#             [target_seq] + states_value)

#         # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_char = reverse_target_char_index[sampled_token_index]
#         decoded_sentence += sampled_char

#         # Exit condition: either hit max length
#         # or find stop character.
#         if (sampled_char == '\n' or
#            len(decoded_sentence) > max_decoder_seq_length):
#             stop_condition = True

#         # Update the target sequence (of length 1).
#         target_seq = np.zeros((1, 1, num_decoder_tokens))
#         target_seq[0, 0, sampled_token_index] = 1.

#         # Update states
#         states_value = [h, c]

#     return decoded_sentence


# for seq_index in range(100):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index: seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     print('-')
#     print('Input sentence:', input_texts[seq_index])
#     print('Decoded sentence:', decoded_sentence)